In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(0.1),
        )
    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self, n_embed, n_head):
        super().__init__()
        self.sa = nn.MultiheadAttention(embed_dim=n_embed, num_heads=n_head, batch_first=True)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ffwd = FeedForward(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        attn_output, _ = self.sa(self.ln1(x), self.ln1(x), self.ln1(x), need_weights=False)
        x = x + attn_output
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTMini(nn.Module):
    def __init__(self, vocab_size, block_size, n_embed=128, n_head=4, n_layer=2):
        super().__init__()
        assert n_embed % n_head == 0
        self.token_embed = nn.Embedding(vocab_size, n_embed)
        self.pos_embed = nn.Embedding(block_size, n_embed)
        self.blocks = nn.Sequential(*[Block(n_embed, n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embed)
        self.head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx):
        B, T = idx.shape
        token_emb = self.token_embed(idx)
        pos = torch.arange(T, device=idx.device)
        pos_emb = self.pos_embed(pos).unsqueeze(0)
        x = token_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        return self.head(x)


In [11]:
# Chargement
checkpoint = torch.load('gpt_mini_checkpoint.pt')

stoi = checkpoint['stoi']
itos = checkpoint['itos']
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

# Recréer le modèle
model = GPTMini(
    vocab_size=checkpoint['vocab_size'],
    block_size=checkpoint['block_size'],
    n_embed=128,
    n_head=4
)
model.load_state_dict(checkpoint['model_state'])
model.eval()

print("✅ Modèle chargé.")

✅ Modèle chargé.


In [12]:
# Générateur de texte
@torch.no_grad()
def generate(model, prompt, max_new_tokens=100, temperature=1.0):
    idx = torch.tensor([encode(prompt)], dtype=torch.long)
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -checkpoint['block_size']:]
        logits = model(idx_cond)
        logits = logits[:, -1, :] / temperature
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        idx = torch.cat([idx, next_id], dim=1)
    return decode(idx[0].tolist())

# Exemple
prompt = "un matin,"
print("\n🔮 Texte généré :\n")
print(generate(model, prompt, max_new_tokens=200, temperature=0.8))


🔮 Texte généré :

un matin,  n n    n    n    n n   n             n                                                                                                                                     c                          
